In [1]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

Using TensorFlow backend.


In [ ]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 0.1  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0]) 

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [ ]:
if __name__ == "__main__":
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)
    agent.load("cartpole-dqn.h5")
    done = False
    batch_size = 32
    episodes = 1000

    for e in range(episodes):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        for time in range(500):
            env.render()
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                print("episode: {}/{}, score: {}, e: {:.2}".format(e, episodes, time, agent.epsilon))
                break
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)
        if e % 10 == 0:
            agent.save("cartpole-dqn.h5")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
episode: 0/1000, score: 413, e: 0.1
episode: 1/1000, score: 447, e: 0.1
episode: 2/1000, score: 470, e: 0.099
episode: 3/1000, score: 280, e: 0.099
episode: 4/1000, score: 183, e: 0.098
episode: 5/1000, score: 166, e: 0.098
episode: 6/1000, score: 163, e: 0.097
episode: 7/1000, score: 148, e: 0.097
episode: 8/1000, score: 214, e: 0.096
episode: 9/1000, score: 178, e: 0.096
episode: 10/1000, score: 183, e: 0.095
episode: 11/1000, score: 152, e: 0.095
episode: 12/1000, score: 162, e: 0.094
episode: 13/1000, score: 137, e: 0.094
episode: 14/1000, score: 236, e: 0.093
episode: 15/1000, score: 178, e: 0.093
episode: 16/1000, score: 221, e: 0.092
episode: 17/1000, score: 177, e: 0.092
episode: 18/1000, score: 354, e: 0.091
episode: 19/1000, score: 209, e: 0.091
episode: 20/1000, score: 400, e: 0.09
episode: 21/1000, score: 448, e: 0.09
episode: 22/1000, score: 300, e: 0.09
episode: 23/1000, sco

episode: 206/1000, score: 167, e: 0.036
episode: 207/1000, score: 196, e: 0.035
episode: 208/1000, score: 242, e: 0.035
episode: 209/1000, score: 199, e: 0.035
episode: 210/1000, score: 171, e: 0.035
episode: 211/1000, score: 154, e: 0.035
episode: 212/1000, score: 181, e: 0.035
episode: 213/1000, score: 205, e: 0.034
episode: 214/1000, score: 175, e: 0.034
episode: 215/1000, score: 177, e: 0.034
episode: 216/1000, score: 175, e: 0.034
episode: 217/1000, score: 176, e: 0.034
episode: 218/1000, score: 180, e: 0.034
episode: 219/1000, score: 187, e: 0.033
episode: 220/1000, score: 173, e: 0.033
episode: 221/1000, score: 203, e: 0.033
episode: 222/1000, score: 185, e: 0.033
episode: 223/1000, score: 316, e: 0.033
episode: 224/1000, score: 206, e: 0.033
episode: 225/1000, score: 227, e: 0.032
episode: 226/1000, score: 245, e: 0.032
episode: 227/1000, score: 280, e: 0.032
episode: 228/1000, score: 151, e: 0.032
episode: 229/1000, score: 164, e: 0.032
episode: 230/1000, score: 153, e: 0.032
